In [26]:
from keras import backend as K
import time
import matplotlib.pyplot as plt
import numpy as np_utils
%matplotlib inline
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D, DepthwiseConv2D, Conv2D, SeparableConv2D, MaxPooling1D, AveragePooling1D
from keras.layers import Input, concatenate
import gensim.downloader as api
from sklearn.model_selection import train_test_split
from keras.layers import Activation, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import GlobalAveragePooling2D
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Nadam, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.regularizers import l2
%config InlineBackend.figure_format='retina'
from keras_contrib.callbacks import CyclicLR
from keras.models import Model
from keras.layers import Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from data_science_utils.vision.keras import *
from time import time
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import missingno as msno
import re
from joblib import Parallel, delayed
from data_science_utils import dataframe as df_utils
from data_science_utils import models as model_utils
from data_science_utils import plots as plot_utils
from data_science_utils.dataframe import column as column_utils
from data_science_utils import misc as misc
from data_science_utils import preprocessing as pp_utils
from data_science_utils import nlp as nlp_utils

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from data_science_utils.dataframe import get_specific_cols

import more_itertools
from more_itertools import flatten
import ast
from sklearn.preprocessing import LabelEncoder

import gc

In [2]:
import sys
import os
sys.path.append(os.getcwd())
from importlib import reload
import lib
reload(lib)
from lib import *

from oclr import OneCycleLR, LRFinder

<module 'lib' from '/home/ec2-user/SageMaker/ML_hackathon_2019/lib.py'>

In [3]:
df_train = pd.read_csv("price_prediction/train.csv")
df_test = pd.read_csv("price_prediction/test.csv")

In [4]:
df_train['text'] = Parallel(n_jobs=16, backend="loky")(delayed(ast.literal_eval)(x) for x in tqdm(df_train['text']))
df_train['text_encoded'] = Parallel(n_jobs=16, backend="loky")(delayed(ast.literal_eval)(x) for x in tqdm(df_train['text_encoded']))
df_train['char_encoded'] = Parallel(n_jobs=16, backend="loky")(delayed(ast.literal_eval)(x) for x in tqdm(df_train['char_encoded']))

df_test['text'] = Parallel(n_jobs=16, backend="loky")(delayed(ast.literal_eval)(x) for x in tqdm(df_test['text']))
df_test['text_encoded'] = Parallel(n_jobs=16, backend="loky")(delayed(ast.literal_eval)(x) for x in tqdm(df_test['text_encoded']))
df_test['char_encoded'] = Parallel(n_jobs=16, backend="loky")(delayed(ast.literal_eval)(x) for x in tqdm(df_test['char_encoded']))


In [4]:
df_train.head()
df_test.head()


,ID,GL,text,text_encoded,char,char_encoded,PRICE
0,1489960,gl_jewelry,"['livsmart', 'resin', 'jewellery', 'display', ...","[28683, 1558, 273, 310, 196, 8, 4, 31, 453, 33...","Livsmart Resin Jewellery Display Stand, 17x11c...","[38, 8, 28, 10, 19, 4, 7, 6, 2, 45, 3, 10, 8, ...",507.62
1,633493,gl_digital_ebook_purchase,"['quantum', 'creation', 'supernatural', 'lurk'...","[7567, 879, 12991, 1, 3900, 207, 267]",Quantum Creation: Does the Supernatural Lurk i...,"[78, 15, 4, 9, 6, 15, 19, 2, 23, 7, 3, 4, 6, 8...",479.90
2,1474591,gl_apparel,"['izod', 'men', 'casual', 'shirt', '_NUM30_', ...","[5244, 24, 35, 32, 127, 26671, 64, 5368, 9, 90...",IZOD Men's Casual Shirt (8907163477392_ZKSH019...,"[41, 81, 46, 39, 2, 29, 3, 9, 63, 10, 2, 23, 4...",829.28
3,830218,gl_apparel,"['dishaa', 'woman', 'rayon', 'line', 'kurta', ...","[45999, 21, 572, 192, 222, 31, 133, 2162, 369,...","Dishaa Women's Rayon A-Line Kurta (Black, X-La...","[39, 8, 10, 13, 4, 4, 2, 42, 5, 19, 3, 9, 63, ...",648.31
4,201144,gl_digital_ebook_purchase,"['return', 'raven', 'ulfrik', 'ormsson', 'saga...","[854, 13891, 1, 1, 7870, 597, 2, 267]",Return of the Ravens (Ulfrik Ormsson's Saga Bo...,"[45, 3, 6, 15, 7, 9, 2, 5, 21, 2, 6, 13, 3, 2,...",332.00


,ID,GL,text,text_encoded,char,char_encoded
0,1585751,gl_apparel,"['folklore', 'woman', 'straight', 'kurta', 'fo...","[10977, 21, 289, 222, 18034, 76, 2997, 1221, 6...",Folklore Women's Straight Kurta (FOKU001013_RE...,"[40, 5, 11, 26, 11, 5, 7, 3, 2, 42, 5, 19, 3, ..."
1,1530678,gl_toy,"['rock', 'party', 'elephant', 'piggy', 'coin',...","[1699, 128, 2210, 5256, 1189, 1552, 170, 312]",Rock The Party Elephant Piggy Coin Bank (Pink)...,"[45, 5, 12, 26, 2, 30, 13, 3, 2, 27, 4, 7, 6, ..."
2,1324955,gl_apparel,"['cherokee', 'unlimited', 'girl', 'shirt', '_N...","[1743, 1043, 82, 32, 1316, 1837, 3727, 4, 4, 1...",Cherokee by Unlimited Girls' T-Shirt (26362639...,"[23, 13, 3, 7, 5, 26, 3, 3, 2, 24, 20, 2, 58, ..."
3,822454,gl_biss,"['packingsupply', 'premium', 'tamper', 'proof'...","[1, 54, 3065, 228, 6192, 71, 4754, 19, 4, 4, 7...",Packingsupply Premium Tamper Proof Courier Bag...,"[27, 4, 12, 26, 8, 9, 16, 10, 15, 18, 18, 11, ..."
4,1604015,gl_shoes,"['contablue', 'funky', 'loafer', '_NUM3_', 'bl...","[15592, 2110, 734, 4, 31, 7, 320, 118, 35, 235...","CONTABLUE Funky Loafers (8 UK, Black)[Material...","[23, 46, 49, 30, 34, 32, 38, 58, 43, 2, 40, 15..."


# GL encodings

In [5]:
from data_science_utils.preprocessing import NeuralCategoricalFeatureTransformer

ct_nn = NeuralCategoricalFeatureTransformer(cols=["GL"],prefix="gl_encoded_",
                                            target_columns=["PRICE"],verbose=0,n_components=16,n_iter=200,)

ct_nn.fit(df_train)

ct_nn.skip_fit = True

Neural Categorical fit start at: 2019-06-13 08:23:04.386101
Instructions for updating:
Colocations handled automatically by placer.


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


Shape of Input to Neural Network: (61, 61), Output shape: (61, 69)
Instructions for updating:
Use tf.cast instead.
Neural Categorical fit done at: 2019-06-13 08:23:40.093439


In [6]:
df_train = ct_nn.transform(df_train)
df_test = ct_nn.transform(df_test)

In [7]:
gl_cols = get_specific_cols(df_train,prefix='gl_')

# Baseline
- GL Mean
- GL Median

In [15]:
df_gl_means = df_train.groupby(["GL"])[['PRICE']].mean().reset_index()

df_results = df_test.merge(df_gl_means, on=["GL"],how="left")
df_results = df_results[["ID","PRICE"]]
df_results["PRICE"] = df_results["PRICE"].fillna(df_results["PRICE"].mean())

df_results.head()

,ID,PRICE
0,1585751,772.612384
1,1530678,1290.724354
2,1324955,772.612384
3,822454,2321.772340
4,1604015,1115.235239


In [16]:
df_results.to_csv("baseline.csv",index=False)

# Baseline 2: Keras Model


For Word Embedding CNN

- Making a text column
    - append GL
    - replace num
    - replace measurement
    
- Hyper Params here:
    - Stopwords
    - word_length_filter
    - lemmatize or not
    - vocab_size in build_dict

In [ ]:
from __future__ import print_function


# set parameters:
max_features = 50000
maxlen = 100
batch_size = 256
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 5

print('Loading data...')

X,y = df_train['text_encoded'].values,df_train['PRICE'].values
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['mae','mse'])
model.count_params()

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))



1159686/1159686 [==============================] - 341s 294us/step - loss: 4033933786.3938 - mean_absolute_error: 894.5309 - val_loss: 16347761.4813 - val_mean_absolute_error: 716.5794
Epoch 2/5
1159686/1159686 [==============================] - 340s 293us/step - loss: 4032204864.3942 - mean_absolute_error: 822.3543 - val_loss: 15832785.5048 - val_mean_absolute_error: 704.4332
Epoch 3/5
1159686/1159686 [==============================] - 339s 292us/step - loss: 4031421547.0617 - mean_absolute_error: 794.2296 - val_loss: 15649164.7560 - val_mean_absolute_error: 698.5270
Epoch 4/5
1159686/1159686 [==============================] - 341s 294us/step - loss: 4030308633.8232 - mean_absolute_error: 781.7042 - val_loss: 15564212.7209 - val_mean_absolute_error: 678.7286
Epoch 5/5
1159686/1159686 [==============================] - 342s 295us/step - loss: 4028990347.3919 - mean_absolute_error: 773.1664 - val_loss: 15462277.1944 - val_mean_absolute_error: 689.7498


In [ ]:
from __future__ import print_function


# set parameters:
max_features = 50000
maxlen = 100
batch_size = 256
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 5

print('Loading data...')

X,X_gl,y = df_train['text_encoded'].values,df_train[gl_cols],df_train['PRICE'].values
x_train, x_test,x_gl_train,x_gl_test, y_train, y_test = train_test_split(X,X_gl, y, test_size=0.2, random_state=42)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)


main_input = Input(shape=(maxlen,), dtype='int32', name='main_input')
x = Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen)(main_input)
x = Dropout(0.2)(x)

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
x = Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1)(x)
# we use max pooling:
x = GlobalMaxPooling1D()(x)
auxiliary_input =  Input(shape=(len(gl_cols),), dtype='float32', name='aux_input')
x = concatenate([x,auxiliary_input])

# We add a vanilla hidden layer:
x = Dense(hidden_dims)(x)
x = Dropout(0.2)(x)
x =Activation('relu')(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
main_output = Dense(1)(x)

model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output])


model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['mae','mse'])
model.count_params()

model.fit([x_train,x_gl_train], y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=([x_test,x_gl_test], y_test))



## With Grouped Conv

In [ ]:
from __future__ import print_function

gc.collect()
# set parameters:
max_features = 50000
maxlen = 100
batch_size = 2048
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 5

print('Loading data...')

X,X_gl,y = df_train['text_encoded'].values,df_train[gl_cols],df_train['PRICE'].values
x_train, x_test,x_gl_train,x_gl_test, y_train, y_test = train_test_split(X,X_gl, y, test_size=0.2, random_state=42)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)


main_input = Input(shape=(maxlen,), dtype='int32', name='main_input')
x = Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen)(main_input)
x = Dropout(0.1)(x)

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:

x = Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1)(x)
xp = x
x = transition_layer(x, n_kernels=64,dropout=0)
x = grouped_layer(x, group_configs=[[dict(n_kernels=64, kernel_size=3, dropout=0.1,dilation_rate=1, padding='same')],
                                    [dict(n_kernels=64, kernel_size=3, dropout=0.1,dilation_rate=2, padding='same')]],
                 out_channels = 64)

x = concatenate([x,xp])
# we use max pooling:
x2 = GlobalAveragePooling1D()(x)
x = GlobalMaxPooling1D()(x)
x = concatenate([x,x2])

auxiliary_input =  Input(shape=(len(gl_cols),), dtype='float32', name='aux_input')
x = concatenate([x,auxiliary_input])

# We add a vanilla hidden layer:
x = Dense(hidden_dims)(x)
x = Dropout(0.2)(x)
x = Activation('relu')(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
main_output = Dense(1)(x)

model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output])

optimizer = Adam(lr=0.0001,)
model.compile(loss='mean_squared_error',
              optimizer=optimizer,
              metrics=['mae','mse'])
model.count_params()

model.fit([x_train,x_gl_train], y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=([x_test,x_gl_test], y_test))



10

Loading data...


2749299

Train on 1159686 samples, validate on 289922 samples
Epoch 1/5
1159686/1159686 [==============================] - 185s 159us/step - loss: 4033730646.5426 - mean_absolute_error: 804.3981 - mean_squared_error: 4033730646.5426 - val_loss: 16842111.5503 - val_mean_absolute_error: 686.5982 - val_mean_squared_error: 16842111.5503
Epoch 3/5
1159686/1159686 [==============================] - 185s 159us/step - loss: 4032106983.2758 - mean_absolute_error: 810.7816 - mean_squared_error: 4032106983.2758 - val_loss: 16408113.0508 - val_mean_absolute_error: 685.7801 - val_mean_squared_error: 16408113.0508
Epoch 4/5
1159686/1159686 [==============================] - 185s 159us/step - loss: 4030989630.1320 - mean_absolute_error: 840.2338 - mean_squared_error: 4030989630.1320 - val_loss: 16202011.9769 - val_mean_absolute_error: 687.1333 - val_mean_squared_error: 16202011.9769
Epoch 5/5
1159686/1159686 [==============================] - 185s 160us/step - loss: 4030299779.8140 - mean_absolute_error: 852.

```
Train on 1159686 samples, validate on 289922 samples
Epoch 1/5
1159686/1159686 [==============================] - 180s 156us/step - loss: 4036435575.9622 - mean_absolute_error: 969.9626 - mean_squared_error: 4036435575.9622 - val_loss: 19007902.2033 - val_mean_absolute_error: 849.3713 - val_mean_squared_error: 19007902.2033
Epoch 2/5
1159686/1159686 [==============================] - 177s 152us/step - loss: 4034018067.3774 - mean_absolute_error: 802.3672 - mean_squared_error: 4034018067.3774 - val_loss: 17264962.1246 - val_mean_absolute_error: 689.0970 - val_mean_squared_error: 17264962.1246
Epoch 3/5
1159686/1159686 [==============================] - 176s 152us/step - loss: 4032383173.0434 - mean_absolute_error: 809.1992 - mean_squared_error: 4032383173.0434 - val_loss: 16797398.9764 - val_mean_absolute_error: 680.6902 - val_mean_squared_error: 16797398.9764
Epoch 4/5
1159686/1159686 [==============================] - 176s 152us/step - loss: 4028829368.1799 - mean_absolute_error: 830.8879 - mean_squared_error: 4028829368.1799 - val_loss: 16374906.9715 - val_mean_absolute_error: 685.6106 - val_mean_squared_error: 16374906.9715
Epoch 5/5
1159686/1159686 [==============================] - 176s 152us/step - loss: 4025341286.7346 - mean_absolute_error: 850.8240 - mean_squared_error: 4025341286.7346 - val_loss: 16511555.2654 - val_mean_absolute_error: 695.2230 - val_mean_squared_error: 16511555.2654
```

In [44]:
from __future__ import print_function

gc.collect()
# set parameters:
max_features = 50000
maxlen = 100
batch_size = 2048
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 5

print('Loading data...')

X,X_gl,y = df_train['text_encoded'].values,df_train[gl_cols],df_train['PRICE'].values
x_train, x_test,x_gl_train,x_gl_test, y_train, y_test = train_test_split(X,X_gl, y, test_size=0.2, random_state=42)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)


main_input = Input(shape=(maxlen,), dtype='int32', name='main_input')
x = Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen)(main_input)
x = Dropout(0.1)(x)

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:

x = Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1)(x)

x1 = MaxPooling1D(pool_size=2)(x)
x2 = AveragePooling1D(pool_size=2)(x)
x = concatenate([x1,x2])

xp = x
x = transition_layer(x, n_kernels=32,dropout=0)
x = conv_layer(x, n_kernels=64,kernel_size=3,dilation_rate=1,padding='same')
x = concatenate([x,xp])
# we use max pooling:
x2 = GlobalAveragePooling1D()(x)
x = GlobalMaxPooling1D()(x)
x = concatenate([x,x2])

auxiliary_input =  Input(shape=(len(gl_cols),), dtype='float32', name='aux_input')
x = concatenate([x,auxiliary_input])

# We add a vanilla hidden layer:
x = Dense(hidden_dims)(x)
x = Dropout(0.2)(x)
x = Activation('relu')(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
main_output = Dense(1)(x)

model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output])

optimizer = Adam(lr=0.0001,)
model.compile(loss='mean_squared_error',
              optimizer=optimizer,
              metrics=['mae','mse'])
model.count_params()

model.fit([x_train,x_gl_train], y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=([x_test,x_gl_test], y_test),shuffle=True)



10

Loading data...


2846875

Train on 1159686 samples, validate on 289922 samples
Epoch 1/5
1159686/1159686 [==============================] - 155s 134us/step - loss: 4036715472.7262 - mean_absolute_error: 983.9760 - mean_squared_error: 4036715472.7262 - val_loss: 19128076.6866 - val_mean_absolute_error: 842.7234 - val_mean_squared_error: 19128076.6866
Epoch 2/5
1159686/1159686 [==============================] - 138s 119us/step - loss: 4034421448.9236 - mean_absolute_error: 778.0972 - mean_squared_error: 4034421448.9236 - val_loss: 17279130.7840 - val_mean_absolute_error: 700.3675 - val_mean_squared_error: 17279130.7840
Epoch 3/5
1159686/1159686 [==============================] - 138s 119us/step - loss: 4032668884.5384 - mean_absolute_error: 802.5082 - mean_squared_error: 4032668884.5384 - val_loss: 16411659.1315 - val_mean_absolute_error: 683.9459 - val_mean_squared_error: 16411659.1315
Epoch 4/5
1159686/1159686 [==============================] - 137s 119us/step - loss: 4031522688.3064 - mean_absolute_error: 849.

```
2772787
Train on 1159686 samples, validate on 289922 samples
Epoch 1/5
1159686/1159686 [==============================] - 177s 152us/step - loss: 4035862812.1876 - mean_absolute_error: 918.4204 - mean_squared_error: 4035862812.1876 - val_loss: 17763249.4091 - val_mean_absolute_error: 738.8896 - val_mean_squared_error: 17763249.4091
Epoch 2/5
1159686/1159686 [==============================] - 159s 137us/step - loss: 4033106637.6809 - mean_absolute_error: 819.1301 - mean_squared_error: 4033106637.6809 - val_loss: 16593876.7654 - val_mean_absolute_error: 692.8858 - val_mean_squared_error: 16593876.7654
Epoch 3/5
1159686/1159686 [==============================] - 159s 137us/step - loss: 4031885271.2714 - mean_absolute_error: 807.6185 - mean_squared_error: 4031885271.2714 - val_loss: 16187226.2574 - val_mean_absolute_error: 689.4169 - val_mean_squared_error: 16187226.2574
Epoch 4/5
1159686/1159686 [==============================] - 159s 137us/step - loss: 4030434700.2492 - mean_absolute_error: 800.9113 - mean_squared_error: 4030434700.2492 - val_loss: 16142123.4253 - val_mean_absolute_error: 686.3089 - val_mean_squared_error: 16142123.4253
Epoch 5/5
1159686/1159686 [==============================] - 159s 137us/step - loss: 4028943375.4024 - mean_absolute_error: 779.0099 - mean_squared_error: 4028943375.4024 - val_loss: 17481870.5746 - val_mean_absolute_error: 1469.6001 - val_mean_squared_error: 17481870.5746
```

In [18]:
X_test = df_test['text_encoded'].values
X_test_gl = df_test[gl_cols]
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
y_preds = model.predict([X_test,X_test_gl])

df_results = df_test[['ID']]
df_results['PRICE'] = y_preds

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [19]:
df_results.head()
df_results.to_csv("baseline-3.csv",index=False)

,ID,PRICE
0,1585751,289.457550
1,1530678,748.071350
2,1324955,289.457550
3,822454,1509.970459
4,1604015,625.523804


# Word Embedding Model

In [39]:
from __future__ import print_function

gc.collect()
# set parameters:
max_features = 50000
maxlen = 100
batch_size = 2048
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 10

print('Loading data...')

X,X_gl,y = df_train['text_encoded'].values,df_train[gl_cols],df_train['PRICE'].values
x_train, x_test,x_gl_train,x_gl_test, y_train, y_test = train_test_split(X,X_gl, y, test_size=0.2, random_state=42)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)


main_input = Input(shape=(maxlen,), dtype='int32', name='main_input')
x = Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen)(main_input)
x = Dropout(0.1)(x)

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:

x = Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1)(x)
xp = x
x = transition_layer(x, n_kernels=64,dropout=0)
xp2 = conv_layer(x,n_kernels=64,kernel_size=3,dilation_rate=1,padding='same')

x = concatenate([xp2, xp])
x = transition_layer(x, n_kernels=64,dropout=0)
xp3 = conv_layer(x,n_kernels=64,kernel_size=3,dilation_rate=2,padding='same')

x = concatenate([xp,xp2,xp3])
# we use max pooling:
x2 = GlobalAveragePooling1D()(x)
x1 = GlobalMaxPooling1D()(x)
x = concatenate([x1,x2])

auxiliary_input =  Input(shape=(len(gl_cols),), dtype='float32', name='aux_input')
x = concatenate([x,auxiliary_input])

# We add a vanilla hidden layer:
x = Dense(hidden_dims)(x)
x = Dropout(0.2)(x)
x = Activation('relu')(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
main_output = Dense(1)(x)

model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output])

optimizer = Adam(lr=0.0001,)
model.compile(loss='mean_squared_error',
              optimizer=optimizer,
              metrics=['mae','mse'])
model.count_params()

model.fit([x_train,x_gl_train], y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=([x_test,x_gl_test], y_test))



2282

Loading data...


2793203

Train on 1159686 samples, validate on 289922 samples
Epoch 1/10
1159686/1159686 [==============================] - 225s 194us/step - loss: 4035866176.8800 - mean_absolute_error: 913.9113 - mean_squared_error: 4035866176.8800 - val_loss: 17918867.4867 - val_mean_absolute_error: 753.0236 - val_mean_squared_error: 17918867.4867
Epoch 2/10
1159686/1159686 [==============================] - 208s 180us/step - loss: 4032961440.3657 - mean_absolute_error: 807.5361 - mean_squared_error: 4032961440.3657 - val_loss: 16587690.5692 - val_mean_absolute_error: 694.2264 - val_mean_squared_error: 16587690.5692
Epoch 3/10
1159686/1159686 [==============================] - 208s 180us/step - loss: 4030689757.3081 - mean_absolute_error: 817.8499 - mean_squared_error: 4030689757.3081 - val_loss: 16222575.0565 - val_mean_absolute_error: 692.6056 - val_mean_squared_error: 16222575.0565
Epoch 4/10
1159686/1159686 [==============================] - 208s 180us/step - loss: 4028827278.6527 - mean_absolute_error: 

# Char Embedding Model

In [ ]:
max_features = 128
maxlen = 500
batch_size = 1024
embedding_dims = 50
epochs = 10


X,X_gl,y = df_train['char_encoded'].values,df_train[gl_cols],df_train['PRICE'].values
x_train, x_test,x_gl_train,x_gl_test, y_train, y_test = train_test_split(X,X_gl, y, test_size=0.2, random_state=42)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

main_input = Input(shape=(maxlen,), dtype='int32', name='main_input')
x = Embedding(output_dim=embedding_dims, input_dim=max_features, input_length=maxlen)(main_input)
x1 = MaxPooling1D(pool_size=5)(x)
x2 = AveragePooling1D(pool_size=5)(x)
x = concatenate([x1,x2])

x = conv_layer(x,n_kernels=128,kernel_size=5, padding='valid')
# x1 = MaxPooling1D(pool_size=2)(x)
# x2 = AveragePooling1D(pool_size=2)(x)
# x = concatenate([x1,x2])
xp1 = x
x = transition_layer(x, n_kernels=64, dropout=0)

x = conv_layer(x,n_kernels=64,kernel_size=15,padding='same')
xp2 = x

x = concatenate([x,xp1])
x = transition_layer(x, n_kernels=32, dropout=0)
x = conv_layer(x,n_kernels=32,kernel_size=15,dilation_rate=2,padding='same')
xp3 = x

x = concatenate([xp2,x])
x = transition_layer(x, n_kernels=32,dropout=0)
x = conv_layer(x,n_kernels=64,kernel_size=15,dilation_rate=1,padding='same')
x = transition_layer(x, n_kernels=32, dropout=0)

x = concatenate([x,xp1,xp3])

x = transition_layer(x, n_kernels=64,dropout=0)


x = pre_dense_layer(x)
K.int_shape(x)
auxiliary_input =  Input(shape=(len(gl_cols),), dtype='float32', name='aux_input')
x_aux = Dense(4)(auxiliary_input)
x = concatenate([x,x_aux])
x = Dense(16)(x)
x = Dropout(0.1)(x)
x =Activation('relu')(x)
main_output = Dense(1)(x)

model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output])



checkpoint = ModelCheckpoint("model.hdf5", monitor='mse', verbose=0, save_best_only=True, mode='max')

optimizer = Adam(0.0001)

callbacks_list = []

model.compile(loss='mean_squared_error',
              optimizer=optimizer,
              metrics=['mae','mse'])
print("Params = ",model.count_params())

model.fit([x_train,x_gl_train], y_train,
          batch_size=batch_size,
          epochs=epochs,callbacks=callbacks_list,
          validation_data=([x_test,x_gl_test], y_test),
         shuffle=True)


# model.load_weights("model.hdf5")


```
Params =  134401
Train on 1159686 samples, validate on 289922 samples
Epoch 1/5
1159686/1159686 [==============================] - 132s 114us/step - loss: 4037083525.2508 - mean_absolute_error: 1044.8966 - mean_squared_error: 4037083525.2508 - val_loss: 19944968.5354 - val_mean_absolute_error: 887.9338 - val_mean_squared_error: 19944968.5354
Epoch 2/5
1159686/1159686 [==============================] - 120s 103us/step - loss: 4036030117.0389 - mean_absolute_error: 873.6337 - mean_squared_error: 4036030117.0389 - val_loss: 18914763.4832 - val_mean_absolute_error: 759.5289 - val_mean_squared_error: 18914763.4832
Epoch 3/5
1159686/1159686 [==============================] - 120s 103us/step - loss: 4035054825.3241 - mean_absolute_error: 855.0202 - mean_squared_error: 4035054825.3241 - val_loss: 18379962.2891 - val_mean_absolute_error: 751.8860 - val_mean_squared_error: 18379962.2891
Epoch 4/5
1159686/1159686 [==============================] - 120s 103us/step - loss: 4034294206.2672 - mean_absolute_error: 907.1308 - mean_squared_error: 4034294206.2672 - val_loss: 18337784.9426 - val_mean_absolute_error: 802.2734 - val_mean_squared_error: 18337784.9426
Epoch 5/5
1159686/1159686 [==============================] - 120s 103us/step - loss: 4031880473.4394 - mean_absolute_error: 926.9110 - mean_squared_error: 4031880473.4394 - val_loss: 23768072.0290 - val_mean_absolute_error: 1974.2186 - val_mean_squared_error: 23768072.0290
```

# Post Processing
- gl min max clipping
- max(0,output)